### Imports

In [ ]:
import pandas as pd
import numpy as np
import ipywidgets as widgets


### Load data

In [ ]:
df = pd.read_csv('../data/name_and_location_with_country.csv')


### Create dictionaries for dropdown values and lookups

In [ ]:
## Create a smaller dataframe with geographic identifiers
df_geo = df[['country','state','county']]
df_geo = df_geo.drop_duplicates()

## Create a country:state dictionary
country_state_df = df_geo[['country','state']]
country_state_df = country_state_df.drop_duplicates()
country_state_dict = country_state_df.groupby('country')['state'].apply(list).to_dict()
#country_state_dict["United Kingdom"]

## Create a state:county dictionary
state_county_df = df_geo[['state','county']]
state_county_df = state_county_df.drop_duplicates()
state_county_dict = state_county_df.groupby('state')['county'].apply(list).to_dict()
#state_county_dict["Northern Ireland"]

## Create a country:county dictionary
country_county_df = df_geo[['country','county']]
country_county_df = country_county_df.drop_duplicates()
country_county_dict = country_county_df.groupby('country')['county'].apply(list).to_dict()
#country_county_dict["United Kingdom"]

## Create a country:state:county dictionary
country_state_county_df = df_geo[['country','state','county']]
country_state_county_df = country_state_county_df.drop_duplicates()
country_state_county_dict = country_state_county_df.groupby(['country','state'])['county'].apply(list).to_dict()
#country_state_county_dict["United Kingdom","England"]


### Create the dropdown widgets

In [ ]:
# Set the original selections
first_country = list(country_state_dict.keys())[0]
first_state = list(state_county_dict.keys())[0]
first_county = state_county_dict[first_state][0]

# Create an object to hold the widgets.Output()
output = widgets.Output()

# Create the dropdown objects
dropdown_country = widgets.Dropdown(options = country_state_dict.keys())
dropdown_state = widgets.Dropdown()
dropdown_county = widgets.Dropdown()

# Create a function to be called by all the dropdowns' eventhandlers
# It clears the old output and re-filters the dataframe using the current values
def common_filtering(country, state, county):
    global output
    output.clear_output()
    with output:
        if ((country != first_country) & (state != first_state) & (county != first_county)):
            # Filter the dataframe with these values
            data = df[(df.country == country) & (df.state == state) & (df.county == county)]
            # Aggregate the results and select top 20
            df2 = data.groupby(['genus','accepted_name']).size().reset_index(name='count').sort_values(['count'], ascending=False).head(20)
        else:
            df2 = df.groupby(['genus','accepted_name']).size().reset_index(name='count').sort_values(['count'], ascending=False).head(20)
        display(df2)   

def dropdown_country_eventhandler(change):
    # Update states list
    dropdown_state.options = sorted(country_state_dict[change.new])
    # Update counties list
    dropdown_county.options = sorted(country_county_dict[change.new])
    # Filter the dataframe
    common_filtering(change.new, dropdown_state.value, dropdown_county.value)
    
def dropdown_state_eventhandler(change):
    # Update counties list
    dropdown_county.options = sorted(country_state_county_dict[dropdown_country.value, change.new])
    # Filter the dataframe
    common_filtering(dropdown_country.value, change.new, dropdown_county.value)

def dropdown_county_eventhandler(change):
    # Filter the dataframe
    common_filtering(dropdown_country.value, dropdown_state.value, dropdown_county.value)

# Wire up the eventhandlers
dropdown_country.observe(dropdown_country_eventhandler, names='value')
dropdown_state.observe(dropdown_state_eventhandler, names='value')
dropdown_county.observe(dropdown_county_eventhandler, names='value')

# Display the widgets
display(dropdown_country)
display(dropdown_state)
display(dropdown_county)


In [ ]:
display(output)
